## USPTO Open Data Portal (ODP) API — Retrieving IP Data

- Reference: [Getting Started — USPTO Open Data APIs](https://data.uspto.gov/apis/getting-started)
- Swagger UI: https://data.uspto.gov/swagger/index.html
---

### **Note:** Before running any subsequent code in this notebook, install the required packages:
- pip install -r requirements.Txt

In [ ]:
# load packages for API calls and data manipulation
import os
import requests
import json
import pandas as pd
from datetime import datetime
import time

# Must be imported BEFORE any Azure SDK or requests usage
import pip_system_certs.wrapt_requests

# load packages for Azure Key Vault to get API key
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

In [ ]:
import time
import logging
import requests
from typing import Optional
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

_logger = logging.getLogger(__name__)
_cache = {}  # simple in-process cache: {(vault_uri, name): (value, expires_at)}

def get_secret_from_keyvault(vault_uri: str, secret_name: str, cache_ttl: int = 300, max_retries: int = 3, backoff_factor: float = 0.5) -> str:
    """
    Retrieve secret from Azure Key Vault with simple caching and retries.
    - vault_uri: e.g. "https://myvault.vault.azure.net/"
    - secret_name: name of the secret
    - cache_ttl: seconds to cache secret in-process
    """
    key = (vault_uri, secret_name)
    now = time.time()
    if key in _cache:
        value, expires_at = _cache[key]
        if now < expires_at:
            return value

    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=vault_uri, credential=credential)

    last_exc: Optional[Exception] = None
    for attempt in range(1, max_retries + 1):
        try:
            secret = client.get_secret(secret_name)
            value = secret.value
            _cache[key] = (value, now + cache_ttl)
            return value
        except Exception as ex:
            last_exc = ex
            wait = backoff_factor * (2 ** (attempt - 1))
            _logger.warning("Failed to read secret (attempt %d/%d). Retrying in %.1fs. (secret=%s, vault=%s) -- not logging value", attempt, max_retries, wait, secret_name, vault_uri)
            time.sleep(wait)

    _logger.error("Unable to retrieve secret %s from %s after %d attempts: %s", secret_name, vault_uri, max_retries, last_exc)
    raise last_exc

In [ ]:

vault = "https://tdg-kv.vault.azure.net/"
secret_name = "USPTO-ApiKey"
api_key = get_secret_from_keyvault(vault, secret_name)
print(f"API Key from Azure Key Vault: {api_key}")